In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui

# 크롬 드라이버 초기화
driver = webdriver.Chrome()
cultures = ["전시회", "박물관", "박물관", "기타", "공원장", "미술관", "문화원"]


addresses = ['중랑구','강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉',
             '동대문','동작','마포','서대문','서초','성동','송파','양천','영등포',
             '용산','은평','종로']

# gangnam 카테고리 초기화
gangnam = {"문화회관": 0, "도서관": 0, "박물관": 0, "기타": 0, "공원장": 0,"미술관": 0, "문화원": 0}
# for i in addresses:
    
    # 데이터 읽기
    data = pd.read_excel(f'filtered_data_중구.xlsx', encoding='utf-8')
    urls = data['홈페이지']
    position = data['문화시설명']

    # CSV 파일에 헤더(컬럼 이름) 기록
    with open(f'count_data_.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # 헤더 작성 (문화시설명 + gangnam의 키 값들)
        header = ['문화시설명'] + list(gangnam.keys())
        writer.writerow(header)

    # 각 홈페이지를 순회하며 텍스트 분석
    for homepage, facility_name in zip(urls, position):
        gangdong = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"": 0, "문화원": 0}
        try: 
            # 셀레니움으로 페이지 요청 및 로드
            driver.get(homepage)
            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            try:
                close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
                for i in len(close_button):
                    time.sleep(0.2)
                    close_button.click()
            except:
                print('닫기 버튼 없음')

            # 페이지의 body 태그에서 모든 텍스트 가져오기
            body_text = driver.find_element(By.TAG_NAME, 'body').text
            
            body_list_1 = body_text.split('\t')
            body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
            body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
            body_set = set(body_list_3)

        except Exception as e:
            print(e)
        try:
            # body_list에서 각 줄을 확인하며 카테고리 찾기
            for body in body_set:
                driver.get('https://aiopen.etri.re.kr/demo/WordRel')
                
                try:
                    word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
                except:
                    word1 = driver.find_element(By.ID,'box3')
                # time.sleep(1)
                try:
                    word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
                except:
                    word2 = driver.find_element(By.ID,'word2')
                # time.sleep(1)

                try:
                    button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
                except:
                    try:
                        button = driver.find_element(By.CLASS_NAME,'btnBlue')
                    except:
                        button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                # time.sleep(1)
                word2.click()
                time.sleep(0.1)
                pyautogui.hotkey('ctrl', 'a')
                pyautogui.hotkey('del')
                word2.send_keys(body)
                # time.sleep(1)
                # 각 장소의 홈페이지에 어떤 키워드가 있는지 분석
                for category in gangnam.keys():
                    word1.click()
                    time.sleep(0.1)
                    pyautogui.hotkey('ctrl', 'a')
                    pyautogui.hotkey('del')
                    word1.send_keys(category)
                    #if category in body:
                    time.sleep(0.1)
                    button.click()
                    time.sleep(0.1)
                    # try:
                    #     value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)
                    # except:
                    #         value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                    # time.sleep(0.1)
                    # if value > 0.5:
                    #     gangnam[category] += 1
                    #     print(gangnam[category])
                    try:
                        value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                    except:
                        value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)    
                    try:
                        value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
                    except:
                        value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
                    time.sleep(0.2)                                     
                    try:
                        if value > 0.2:
                            gangnam[category] += 1
                            print(gangnam[category])
                    except:
                        if value_2 > 0:
                            gangnam[category] += 1
                            print(gangnam[category])
                    time.sleep(0.1)
        except Exception as e:
            print(e)
                    # 데이터 CSV 파일에 기록하기
    with open(f'count_data_{i}.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # gangnam 값을 리스트로 변환하고 position과 결합
        row = [facility_name] + list(gangnam.values())
        writer.writerow(row)
        
            
    # 결과 출력
    print(gangnam)

    # 크롬 드라이버 종료
    driver.quit()


닫기 버튼 없음
1
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]"}
  (Session info: chrome=128.0.6613.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF739DEB632+29090]
	(No symbol) [0x00007FF739D5E6E9]
	(No symbol) [0x00007FF739C1B1CA]
	(No symbol) [0x00007FF739C6EFD7]
	(No symbol) [0x00007FF739C6F22C]
	(No symbol) [0x00007FF739CB97F7]
	(No symbol) [0x00007FF739C9672F]
	(No symbol) [0x00007FF739CB65D9]
	(No symbol) [0x00007FF739C96493]
	(No symbol) [0x00007FF739C609B1]
	(No symbol) [0x00007FF739C61B11]
	GetHandleVerifier [0x00007FF73A10883D+3294125]
	GetHandleVerifier [0x00007FF73A154423+3604371]
	GetHandleVerifier [0x00007FF73A14A2E7+3563095]
	GetHandleVerifier [0x00007FF739EA6F16+797318]
	(No symbol) [0x00007FF739D6986F]
	(No symbol) [0x00007FF739D65454]
	(No symbol

In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui

# 크롬 드라이버 초기화
driver = webdriver.Chrome()
cultures = ["문화회관", "도서관", "박물관", "기타", "공원장", "미술관", "문화원"]


addresses = ['중랑구','강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉',
             '동대문','동작','마포','서대문','서초','성동','송파','양천','영등포',
             '용산','은평','종로']

# gangnam 카테고리 초기화
# gangnam = {"문화회관": 0, "도서관": 0, "박물관": 0, "기타": 0, "공원장": 0,"미술관": 0, "문화원": 0}
cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
# for i in addresses:
    
# 데이터 읽기
data = pd.read_excel('seoul_culture_place_junggu.xlsx')
# data = pd.read_excel('filtered_data_중랑구.xlsx')
urls = data['홈페이지']
position = data['문화시설명']

# CSV 파일에 헤더(컬럼 이름) 기록
with open(f'count_data_중구.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 작성 (문화시설명 + gangnam의 키 값들)
    header = ['문화시설명'] + list(cultures_dic.keys())
    writer.writerow(header)

# 각 홈페이지를 순회하며 텍스트 분석
for homepage, facility_name in zip(urls, position):
    cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
    print(position, end= ' ')
    try: 
        # 셀레니움으로 페이지 요청 및 로드
        driver.get(homepage)
        WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
        try:
            close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
            for i in len(close_button):
                time.sleep(0.2)
                close_button.click()
        except:
            print('닫기 버튼 없음')

        # 페이지의 body 태그에서 모든 텍스트 가져오기
        body_text = driver.find_element(By.TAG_NAME, 'body').text
        
        body_list_1 = body_text.split('\t')
        body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
        body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
        body_set = set(body_list_3)
        print(len(body_set))
        print(position, end= ' ')
    except Exception as e:
        print(e)
    try:
        # body_list에서 각 줄을 확인하며 카테고리 찾기
        for body in body_set:
            driver.get('https://aiopen.etri.re.kr/demo/WordRel')
            
            try:
                word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
            except:
                word1 = driver.find_element(By.ID,'box3')
            # time.sleep(1)
            try:
                word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
            except:
                word2 = driver.find_element(By.ID,'word2')
            # time.sleep(1)

            try:
                button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
            except:
                try:
                    button = driver.find_element(By.CLASS_NAME,'btnBlue')
                except:
                    button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            # time.sleep(1)
            word2.click()
            time.sleep(0.1)
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.hotkey('del')
            word2.send_keys(body)
            # time.sleep(1)

            # 각 장소의 홈페이지에 어떤 키워드가 있는지 분석
            for category in cultures_dic.keys():
                word1.click()
                time.sleep(0.2)
                pyautogui.hotkey('ctrl', 'a')
                time.sleep(0.2)
                pyautogui.hotkey('del')
                word1.send_keys(category)
                #if category in body:
                # time.sleep(0.2)
                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                button.click()
                time.sleep(0.1)

                try:
                    value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                except:
                    value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)    
                try:
                    value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
                except:
                    value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
                time.sleep(0.2)                                     
                try:
                    if value > 0.2:
                        cultures_dic[category] += 1
                        print(cultures_dic[category],end= ' ')
                except:
                    if value_2 > 0:
                        cultures_dic[category] += 1
                        print(cultures_dic[category], end=' ')
                time.sleep(0.1)
    except Exception as e:
        print(e)
                # 데이터 CSV 파일에 기록하기
    with open(f'count_data_중구.csv', 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # gangnam 값을 리스트로 변환하고 position과 결합
        row = [facility_name] + list(cultures_dic.values())
        writer.writerow(row)
    
# 결과 출력
# print)

# 크롬 드라이버 종료
driver.quit()


0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4          중랑구민회관
5      옹기테마공원 체험장
6          중랑아트센터
7          중랑장미공원
8           중랑문화원
9     중랑구립면목정보도서관
10         용마폭포공원
11           중랑구청
12        중랑상봉도서관
13          중랑캠핑숲
Name: 문화시설명, dtype: object 닫기 버튼 없음
176
0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4          중랑구민회관
5      옹기테마공원 체험장
6          중랑아트센터
7          중랑장미공원
8           중랑문화원
9     중랑구립면목정보도서관
10         용마폭포공원
11           중랑구청
12        중랑상봉도서관
13          중랑캠핑숲
Name: 문화시설명, dtype: object Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]"}
  (Session info: chrome=128.0.6613.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF69C9DB632+29090]
	(No symbol) [0x00007FF69C94E6E9]
	(No symbol)

In [3]:
import urllib3
import json
# API 요청 URL
openApiURL = "http://aiopen.etri.re.kr:8000/ParaphraseQA"

# ETRI에서 발급받은 Access Key
accessKey = "5d059aa9-b7b7-483a-ba61-e3572b9693e0"

# 분석할 문장들 예시 (전통과 한옥 관련 문장)
sentence1 = "귤"
sentence2 = '사과'

# API 요청 데이터 설정
requestJson = {
    "argument": {
        "sentence1": sentence1,
        "sentence2": sentence2
    }
}

# HTTP 통신 설정 및 요청
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8", "Authorization": accessKey},
    body=json.dumps(requestJson)
)

# 응답 결과 출력
print("[Response Code] " + str(response.status))
print("[Response Body]")
print(str(response.data, "utf-8"))
analize = set(str(response.data, "utf-8"))


[Response Code] 200
[Response Body]
{"result":"0","return_type":"com.google.gson.internal.LinkedTreeMap","return_object":{"result":"not paraphrase"}}


In [13]:
import urllib3
import json

# API URL 및 Access Key
openApiURL = "http://aiopen.etri.re.kr:8000/WiseWWN/WordRel"
accessKey = "5d059aa9-b7b7-483a-ba61-e3572b9693e0"  # 발급받은 API 키

# 분석할 두 단어와 의미 ID
firstWord = "굴"
firstSenseId = "AM"  # 실제 의미 ID로 교체
secondWord = "전복"
secondSenseId = "AM"  # 실제 의미 ID로 교체

# 요청할 데이터
requestJson = {
    "argument": {
        "first_word": firstWord,
        "first_sense_id": firstSenseId,
        "second_word": secondWord,
        "second_sense_id": secondSenseId
    }
}

# HTTP 요청 보내기
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={
        "Content-Type": "application/json; charset=UTF-8",
        "Authorization": accessKey
    },
    body=json.dumps(requestJson)
)

# 응답 출력
print("[responseCode] " + str(response.status))
print("[responBody]")
print(response.data.decode("utf-8"))


[responseCode] 200
[responBody]
{"result":0,"return_object":{"MetaInfo":{"Title":"","Link":""},"WWN WordRelInfo":{"FirstWordInfo":{"Word":"굴","HomonymCode":0,"PolysemyCode":0,"Definition":"사새목 굴과에 속하는연체동물의 총칭.","POS":"u"},"SecondWordInfo":{"Word":"전복","HomonymCode":0,"PolysemyCode":0,"Definition":"원시복족목(原始腹族目) 전복과에 딸린 연체동물의 총칭.","POS":"u"},"WordRelInfo":{"ShortedPath":["굴_0000","LCS: root","전복_0000"],"Distance":2,"Similarity":[{"Algorithm":"ETRI","SimScore":0},{"Algorithm":"Resnik","SimScore":0},{"Algorithm":"Lin","SimScore":0},{"Algorithm":"Jiang and Conrath","SimScore":0},{"Algorithm":"Leacock and Chodorow","SimScore":0},{"Algorithm":"Wu & Palmer","SimScore":0},{"Algorithm":"Hirst and St-Onge","SimScore":0},{"Algorithm":"Banerjee and Pedersen","SimScore":0},{"Algorithm":"Patwardhan","SimScore":0},{"Algorithm":"Pekar et al","SimScore":0},{"Algorithm":"Lin + GraSM","SimScore":0}]}}}}


In [35]:
gangnam = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
print(gangnam[0]["문화회관"])
gangdong = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
gangbuk = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
gangseo = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
gwanak = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
guro = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
gwangjin = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
geumcheon = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
nowon = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
dobong = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
dongdaemun = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
dongjak = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
mapo = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
seodaemun = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
seocho = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
seongdong = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
seongbuk = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
songpa = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
yangcheon = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
yeongdeungpo = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
yongsan = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
eunpyeong = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
jongno = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
jung = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
jungnang = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
cultures = ["문화회관", "도서관", "박물관","기타" ,"공원장" ,"미술관", "문화원"]

print(gangdong[1]['도서관'])

0
0


In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui
import datetime
now = datetime.datetime.now
print(now)
# 크롬 드라이버 초기화
driver = webdriver.Chrome()
cultures = ["문화회관", "도서관", "박물관", "기타", "공원장", "미술관", "문화원"]

addresses = ['중랑구','강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉',
             '동대문','동작','마포','서대문','서초','성동','송파','양천','영등포',
             '용산','은평','종로']

cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}

# 데이터 읽기
data = pd.read_csv('filtered_data_중랑구.csv', encoding='utf-8')
urls = data['홈페이지']
position = data['문화시설명']

# CSV 파일에 헤더(컬럼 이름) 기록
with open(f'count_data_중랑구.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # 헤더 작성 (문화시설명 + gangnam의 키 값들)
    header = ['문화시설명'] + list(cultures_dic.keys())
    writer.writerow(header)

# 각 홈페이지를 순회하며 텍스트 분석
for homepage, facility_name in zip(urls, position):
    cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
    print(position, end= ' ')
    body_list_4 = []
    body_list_2 = []
    body_list_1 = []
    body_list_3 = []    
    body_set = {}
    body_list_5 = {}
    # try: 
    # 셀레니움으로 페이지 요청 및 로드
    driver.get(homepage)
    WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
    try:
        close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
        close_button.click()
    except:
        print('닫기 버튼 없음')

    # 페이지의 body 태그에서 모든 텍스트 가져오기
    body_text = driver.find_element(By.TAG_NAME, 'body').text
    
    body_list_1 = body_text.split('\t')
    body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
    body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
    for i in body_set:
        if 1 < len(i) < 10:
            body_list_4.append(i)
    body_set = set(body_list_4)  # Set을 리스트로 변환하여 인덱스 접근 가능
    body_list_5 = list(body_set)
    # 리스트를 4부분으로 나누기
    n = len(body_list_5)
    chunk_size = n // 4
    
    body_parts = [
        body_list_5[:chunk_size],
        body_list_5[chunk_size:2*chunk_size],
        body_list_5[2*chunk_size:3*chunk_size],
        body_list_5[3*chunk_size:]
    ]
    
    print(len(body_set))
    print(position, end= ' ')
# except Exception as e:
#     print(e)

# try:
    # 각 부분을 순회하면서 텍스트 분석
    for part in body_parts:
        for body in part:
            driver.get('https://aiopen.etri.re.kr/demo/WordRel')
            
            try:
                word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
            except:
                word1 = driver.find_element(By.ID,'box3')
            
            try:
                word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
            except:
                word2 = driver.find_element(By.ID,'word2')

            try:
                button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
            except:
                try:
                    button = driver.find_element(By.CLASS_NAME,'btnBlue')
                except:
                    button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            
            word2.click()
            time.sleep(0.2)
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.hotkey('del')
            time.sleep(0.2)
            word2.send_keys(body)

            for category in cultures_dic.keys():
                word1.click()
                time.sleep(0.2)
                pyautogui.hotkey('ctrl', 'a')
                time.sleep(0.2)
                pyautogui.hotkey('del')
                time.sleep(0.2)
                word1.send_keys(category)
                
                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                button.click()
                time.sleep(0.1)

                try:
                    value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                except:
                    value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)    
                try:
                    value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
                except:
                    value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
                time.sleep(0.2)                                     
                try:
                    if value > 0.2:
                        cultures_dic[category] += 1
                        print(cultures_dic[category], end=' ')
                except:
                    if value_2 > 0:
                        cultures_dic[category] += 1
                        print(cultures_dic[category], end=' ')
                time.sleep(0.1)
            # 데이터 CSV 파일에 기록하기
        with open(f'count_data_중랑구.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            # gangnam 값을 리스트로 변환하고 position과 결합
            row = [facility_name] + list(cultures_dic.values())
            writer.writerow(row)
# except Exception as e:
#     print(e)

    
    
# 크롬 드라이버 종료
driver.quit()


<built-in method now of type object at 0x00007FFFAADA7640>
0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4          중랑구민회관
5      옹기테마공원 체험장
6          중랑아트센터
7          중랑장미공원
8           중랑문화원
9     중랑구립면목정보도서관
10         용마폭포공원
11           중랑구청
12        중랑상봉도서관
13          중랑캠핑숲
Name: 문화시설명, dtype: object 닫기 버튼 없음
0
0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4          중랑구민회관
5      옹기테마공원 체험장
6          중랑아트센터
7          중랑장미공원
8           중랑문화원
9     중랑구립면목정보도서관
10         용마폭포공원
11           중랑구청
12        중랑상봉도서관
13          중랑캠핑숲
Name: 문화시설명, dtype: object 0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4          중랑구민회관
5      옹기테마공원 체험장
6          중랑아트센터
7          중랑장미공원
8           중랑문화원
9     중랑구립면목정보도서관
10         용마폭포공원
11           중랑구청
12        중랑상봉도서관
13          중랑캠핑숲
Name: 문화시설명, dtype: object 닫기 버튼 없음
0
0        중랑구평생학습관
1         중랑문화체육관
2       중랑구립정보도서관
3        중랑구민체육센터
4         

TimeoutException: Message: 


In [11]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui
# datetime 모듈을 불러옵니다.
import datetime

# 현재 시간을 얻습니다.
현재시간 = datetime.datetime.now()

# 현재 시간을 출력합니다.
print("현재 시간은:", 현재시간)

# 크롬 드라이버 초기화
# gangnam = [{"문화회관" : 0}, {"도서관" : 0}, {"박물관" : 0}, {"기타" : 0}, {"공원장" : 0}, {"미술관" : 0}, {"문화원" : 0}]
cultures_dic = {"문화회관": 0, "도서관": 0, "박물관": 0, "기타": 0, "공원장": 0, "미술관": 0, "문화원": 0}
cultures_list = ["문화회관", "도서관", "박물관", "기타", "공원장", "미술관", "문화원"]
driver = webdriver.Chrome()
driver.set_window_size(600, 600)
position = driver.get_window_position()
x = position.get('47')
y = position.get('47')
try:
    driver.get('https://www.jungnang.go.kr/study/main.do')
    body_set_2 = {}
    body_list_4 = []
    WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
    try:
        close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
        for i in len(close_button):
            time.sleep(0.2)
            close_button.click()
    except:
        print('닫기 버튼 없음')

    # 페이지의 body 태그에서 모든 텍스트 가져오기
    body_text = driver.find_element(By.TAG_NAME, 'body').text
    
    body_list_1 = body_text.split('\t')
    body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
    body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
    body_set = set(body_list_3)
    for i in body_set:
        if 1 < len(i) < 10:
            body_list_4.append(i)
    body_set = set(body_list_4)  # Set을 리스트로 변환하여 인덱스 접근 가능
    body_list_5 = list(body_set)
    # 리스트를 4부분으로 나누기
    n = len(body_list_5)
    chunk_size = n // 4
    
    body_parts = [
        body_list_5[:chunk_size],
        body_list_5[chunk_size:2*chunk_size],
        body_list_5[2*chunk_size:3*chunk_size],
        body_list_5[3*chunk_size:]
    ]
    for i in body_set:
        if 1 < len(i) < 10:
            body_list_4.append(i)
    body_set_2 = body_list_4
    print(len(body_set_2))
    print(body_set_2)
except Exception as e:
    print(e)
try:
    # body_list에서 각 줄을 확인하며 카테고리 찾기
    for body in body_set_2:
        driver.get('https://aiopen.etri.re.kr/demo/WordRel')
        try:
            word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
        except:
            word1 = driver.find_element(By.ID,'box3')
        # time.sleep(1)
        try:
            word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
        except:
            word2 = driver.find_element(By.ID,'word2')
        # time.sleep(1)

        try:
            button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
        except:
            try:
                button = driver.find_element(By.CLASS_NAME,'btnBlue')
            except:
                button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

        WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
        # time.sleep(1)
        word2.click()
        time.sleep(0.2)
        pyautogui.hotkey('ctrl', 'a')
        time.sleep(0.2)
        pyautogui.hotkey('del')
        time.sleep(0.2)
        word2.send_keys(body)
        time.sleep(0.2)
        # time.sleep(1)
        # 각 장소의 홈페이지에 어떤 키워드가 있는지 분석
        for category in cultures_dic.keys():
            word1.click()
            time.sleep(0.2)
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.hotkey('del')
            time.sleep(0.2)
            word1.send_keys(category)
            time.sleep(0.2)
            #if category in body:
            button.click()
            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            time.sleep(0.2)
            try:
                value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
            except:
                try:
                    value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)
                except Exception as e:
                    print(e)

            # try:
            #     value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
            # except:
            #     try:
            #         value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
            #     except Exception as e:
            #         print(e)
                
            time.sleep(0.1)                                     
            # try:
            if value > 0.2 :
                cultures_dic[category] += 1
                print(cultures_dic[category])
            # except:
            #     if value_2 > 0:
            #         cultures_dic[category] += 1
            #         print(cultures_dic[category])
            time.sleep(0.1)
except Exception as e:
    print(e)    
    # 데이터 CSV 파일에 기록하기
with open(f'count_data_중랑구.csv', 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    # gangnam 값을 리스트로 변환하고 position과 결합
    row = list(cultures_dic.values())
    writer.writerow(row)

# 현재 시간을 얻습니다.
현재시간 = datetime.datetime.now()

# 현재 시간을 출력합니다.
print("현재 시간은:", 현재시간)

현재 시간은: 2024-08-31 21:41:41.421900
닫기 버튼 없음
262
['이전', '서울특별시', 'rights', '평생학습계좌제', '접수중', '포털', '10,000원', '역사', '교육', '평생교육진흥원', '프로그램(맛있는', '중랑교육종합포털', '1/8', 'Previous', '4064남성', '망우로', '다과(떡)', '©2021', '모델워킹', '로그인', '예쁜', 'E-BOOK', '수강생', '평생학습:', '재료비', '안내', '8월', '국가평생교육진흥원', '수목', '꽃보다', '평생학습관', '[서일]', '공지사항', '자치회관', '누구나', '[디지털', '있는', 'reserved', '[모두랑]', '방정환', '여름학기', '[그리다]', '평생학습', 'TEL', '탐방', '화수', '교실', '특강', '수제', '자동재생', '교육지원센터', '포토', '찾아가는', '배움과', '다음', '3기(활용)', '무료', '마사지', '충전)', '배워봅시다', '더보기', 'All', '[나비우채]', '[꼼지락]', '바로가기', '슬라이드로', '서예', '중랑구', '바르게', '생활', '뜨는', '참여자', '서울시', '모집', 'FAX', '나만의', '67길', '2024년', '배울', '어디서나', '자수를', 'Next', '중급-여름학기', '주메뉴', '[쏙쏙]', '10(망우동)', '언제,', '대상', '만들기', '천연비누', '열린', '동네배움터', '[평생학습관]', '월화', '요리', '즐거운', '9월', '학습의', '가을학기', '프로그램', '중랑', '생활문해]스마트폰', '걷기', '갤러리', '교육시간', '코바늘로', '소품', '보타니컬아트', '[02059]', 'Copyright', '마감임박', '50,000원', '중랑의', 'Office.', '일시정지', '통기타', '수강료', '48,000원', '나눔의', '[신바람

In [9]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
import nltk

# NLTK의 punkt 데이터 다운로드
nltk.download('punkt')

# 웹 페이지의 URL
url = 'https://www.museum.go.kr/site/main/home'

# 웹 페이지 요청 및 파싱
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 페이지에서 텍스트 추출
text = soup.get_text()

# 텍스트를 단어 단위로 토큰화
words = word_tokenize(text)

# 관심 있는 카테고리 리스트
categories = [
    '전시회', '박물관', '음악', '무용', '전통예술', 
    '연극공연', '영화관람', '대중공연'
]

# 카테고리와 관련된 텍스트 비율 계산
def calculate_category_proportions(text, categories):
    category_counts = {category: 0 for category in categories}
    total_words = len(words)
    
    for category in categories:
        category_counts[category] = text.lower().count(category)
    
    proportions = {category: count / total_words for category, count in category_counts.items()}
    
    return proportions

# 카테고리 비율 계산
category_proportions = calculate_category_proportions(text, categories)

# 결과 출력
print("카테고리 비율:")
for category, proportion in category_proportions.items():
    print(f"{category}: {proportion:.4f}")

# TF-IDF 벡터화 및 분석
def get_tfidf_scores(text, categories):
    vectorizer = TfidfVectorizer(vocabulary=categories)
    X = vectorizer.fit_transform([text])
    feature_names = vectorizer.get_feature_names_out()
    
    scores = {category: X[0, i] for i, category in enumerate(feature_names)}
    
    return scores

# TF-IDF 점수 계산
category_scores = get_tfidf_scores(text, categories)

# 결과 출력
print("\n카테고리 TF-IDF 점수:")
for category, score in category_scores.items():
    print(f"{category}: {score:.4f}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vvv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\vvv/nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\vvv\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/nltk_data'
**********************************************************************


In [7]:
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy

# spacy 모델 로드
nlp = spacy.load('ko_core_news_sm')

# 웹 페이지의 URL
url = 'https://www.museum.go.kr/site/main/home'

# 웹 페이지 요청 및 파싱
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 페이지에서 텍스트 추출
text = soup.get_text()

# 문장을 나누기 (spacy 사용)
def split_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

sentences = split_sentences(text)

# 관심 있는 카테고리 리스트
categories = [
    '전시회', '박물관', '음악', '무용', '전통예술', 
    '연극공연', '영화관람', '대중공연'
]

# TF-IDF 벡터화 및 분석
def get_tfidf_scores(sentences, categories):
    vectorizer = TfidfVectorizer(vocabulary=categories)
    X = vectorizer.fit_transform(sentences)
    feature_names = vectorizer.get_feature_names_out()
    
    scores = {category: [] for category in categories}
    
    for i, sentence in enumerate(sentences):
        tfidf_scores = X[i].toarray()[0]
        for j, category in enumerate(categories):
            score = tfidf_scores[j]
            if score > 0:
                scores[category].append((i, score, sentences[i]))
    
    return scores

# TF-IDF 점수 계산
category_scores = get_tfidf_scores(sentences, categories)

# 결과 출력
for category, score_data in category_scores.items():
    print(f"\n{category}:")
    if score_data:
        for index, score, sentence in sorted(score_data, key=lambda x: -x[1]):
            print(f"문장: {sentence}")
            print(f"점수: {score:.4f}")
    else:
        print("관련된 문장이 없습니다.")



전시회:
관련된 문장이 없습니다.

박물관:
문장: 국립중앙박물관




















본문 바로가기











오늘하루보지않기

배너닫기







어린이박물관


학예사자격증


박물관신문


소속박물관 바로가기  

국립경주박물관
국립광주박물관
국립전주박물관
국립대구박물관
국립부여박물관
국립공주박물관
국립진주박물관
국립청주박물관
국립김해박물관
국립제주박물관
국립춘천박물관
국립나주박물관
국립익산박물관




Language 

English
日本語
中文















































관람 정보관람 안내전시 해설전시 해설 안내전시 해설 예약큐레이터와의 대화예약하기어린이박물관 관람 예약VR체험 예약 (디지털 실감 영상관 2)교육 프로그램 예약학생 단체 관람 예약전시 해설 예약외부해설 예약시설 대관 예약야간개장 셔틀버스 예약관람·편의시설박물관 전체 안내전시 공간주요 시설야외 시설식당, 편의시설편의시설 연락처층별 안내접근성 안내오시는 길지하철버스자기차량주차 안내전시상설 전시상설전시실 층별 안내추천 동선선사·고대관중·근세관서화관기증관사유의 방조각·공예관세계문화관역사의 길야외 전시특별 전시현재 전시예정 전시지난 전시디지털 실감 영상관디지털 실감 영상관 안내실감1관실감2관실감3관디지털 광개토대왕릉비경천사 십층석탑명품 실감열린마당 실감 전광판온라인 실감콘텐츠메타버스 박물관 '힐링 동산'공간 오감온라인 전시관우리문화유산 국외 전시소속 국립박물관 전시전시도록외국박물관 소장 한국문화유산미주유럽아시아교육일반교육(MODU)전문교육(뮤지엄 아카데미)문화행사행사공연소장품큐레이터 추천 소장품소장품 검색국보·보물 검색3D 데이터 검색수어 동영상음성 안내소장품 열람·복제소장품 대여 안내문화유산 기증 안내기증안내기증내역/사례학술행사·출판학술행사정기간행물간행물 검색기획특별전 전시보고서논문 투고고고학조사 연구 현황발굴 보고서연구 보고서학술 토론회 자료역사학조사 연구 현황외규장각 의궤 총서연구 보고서학술 토론회 자료

In [3]:
print(nltk.data.path)

['C:\\Users\\vvv/nltk_data', 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data', 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data', 'c:\\Users\\vvv\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data', 'C:\\Users\\vvv\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
import pyautogui

# 크롬 드라이버 초기화
driver = webdriver.Chrome()
cultures_dic = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"대중공연": 0, "문화원": 0,'운동경기': 0,}
cultures_list = ['전시회', '음악 및 무용발표회', '박물관', '전통예술공연', '연극공연', '대중공연', '문화원', '운동경기']

addresses = ['중랑구','강남','강동','강북','강서','관악','광진','구로','금천','노원','도봉',
             '동대문','동작','마포','서대문','서초','성동','송파','양천','영등포',
             '용산','은평','종로']

# gangnam 카테고리 초기화
# gangnam = {"문화회관": 0, "도서관": 0, "박물관": 0, "기타": 0, "공원장": 0,"미술관": 0, "문화원": 0}
for i in addresses:
    
    # 데이터 읽기
    data = pd.read_excel(f'filtered_data_{i}.xlsx', encoding='utf-8')
    urls = data['홈페이지']
    position = data['문화시설명']

    # CSV 파일에 헤더(컬럼 이름) 기록
    with open(f'count_data_{i}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        # 헤더 작성 (문화시설명 + gangnam의 키 값들)
        header = ['문화시설명'] + list(gangnam.keys())
        writer.writerow(header)

    # 각 홈페이지를 순회하며 텍스트 분석
    for homepage, facility_name in zip(urls, position):
        gangdong = {"전시회": 0, "음악 및 무용발표회": 0, "박물관": 0, "전통예술공연": 0, "연극공연": 0,"": 0, "문화원": 0}
        try: 
            # 셀레니움으로 페이지 요청 및 로드
            driver.get(homepage)
            WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
            try:
                close_button = driver.find_element(By.XPATH, '//*[text()="닫기"]')
            except:
                close_button = driver.find_element(By.XPATH, '//*[text()="오늘하루"]')
                for i in len(close_button):
                    time.sleep(0.2)
                    close_button.click()

            # 페이지의 body 태그에서 모든 텍스트 가져오기
            body_text = driver.find_element(By.TAG_NAME, 'body').text
            
            body_list_1 = body_text.split('\t')
            body_list_2 = [item for sublist in body_list_1 for item in sublist.split(' ')]
            body_list_3 = [item for sublist in body_list_2 for item in sublist.split('\n')]
            body_set = set(body_list_3)

        except Exception as e:
            print(e)
        # try:
            # body_list에서 각 줄을 확인하며 카테고리 찾기
            for body in body_set:
                driver.get('https://aiopen.etri.re.kr/demo/WordRel')
                
                try:
                    word1 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[1]/td/input')
                except:
                    word1 = driver.find_element(By.ID,'box3')
                # time.sleep(1)
                try:
                    word2 = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[2]/table/tbody/tr[2]/td/input')
                except:
                    word2 = driver.find_element(By.ID,'word2')
                # time.sleep(1)

                try:
                    button = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]/button')
                except:
                    try:
                        button = driver.find_element(By.CLASS_NAME,'btnBlue')
                    except:
                        button = driver.find_element(By.CSS_SELECTOR,'btnBlue')

                WebDriverWait(driver, 10).until(lambda driver: driver.execute_script("return document.readyState") == "complete")
                # time.sleep(1)
                word2.click()
                time.sleep(0.1)
                pyautogui.hotkey('ctrl', 'a')
                pyautogui.hotkey('del')
                word2.send_keys(body)
                # time.sleep(1)
                # 각 장소의 홈페이지에 어떤 키워드가 있는지 분석
                for category in gangnam.keys():
                    word1.click()
                    time.sleep(0.1)
                    pyautogui.hotkey('ctrl', 'a')
                    pyautogui.hotkey('del')
                    word1.send_keys(category)
                    #if category in body:
                    time.sleep(0.1)
                    button.click()
                    time.sleep(0.1)
                    # try:
                    #     value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)
                    # except:
                    #         value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                    # time.sleep(0.1)
                    # if value > 0.5:
                    #     gangnam[category] += 1
                    #     print(gangnam[category])
                    try:
                        value = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[2]/tbody/tr/td[1]').text)
                    except:
                        value = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[2]/tbody/tr/td[1]').text)    
                    try:
                        value_2 = float(driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[5]/table[1]/tbody/tr/td[2]').text)
                    except:
                        value_2 = float(driver.find_element(By.XPATH, '//*[@id="api_result_box_3"]/table[1]/tbody/tr/td[2]').text)
                    time.sleep(0.2)                                     
                    try:
                        if value > 0.2:
                            gangnam[category] += 1
                            print(gangnam[category])
                    except:
                        if value_2 > 0:
                            gangnam[category] += 1
                            print(gangnam[category])
                    time.sleep(0.1)
        with open(f'count_data_{i}.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            # gangnam 값을 리스트로 변환하고 position과 결합
            row = [facility_name] + list(gangnam.values())
            writer.writerow(row)
        # except Exception as e:
        #     print(e)
                    # 데이터 CSV 파일에 기록하기
    
        
            
    # 결과 출력
    print(gangnam)

    # 크롬 드라이버 종료
    driver.quit()
